In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

* This notebook contains the complete code to extract the online news related to an specific firm, in this case Microsoft (MSFT).

    1. All the news have been extracted from https://business.financialpost.com/

In [23]:
import requests
import urllib
import sys
import numpy  as np
import pandas as pd
import json
from pyquery import pyquery
from bs4 import BeautifulSoup

# 1. Extract web links according to a signature and the number of pages


In [27]:
def get_internal_links(ur):
    info  = BeautifulSoup(requests.get(ur,allow_redirects = True).content,'html.parser').find_all("h4",{"class":"entry-title"})
    links  = [ a['href'] for each_link in info for a in each_link.find_all('a', href=True)]
    print(f'links: {links}')
    return(links)   

def get_links(url):
    pq = pyquery.PyQuery(url)
    seen = set()
    for a in pq.find('a'):
        href = a.get('href')
        if href.startswith('/pmn/business-pmn'):
            seen.add(href)
    print(f'hrefs for {url}:\n{json.dumps(list(seen), indent=4)}')
    return seen

In [28]:
def extract_link_of_news(k_word,n_of_page):
    news_df                  = pd.DataFrame()
#     link =  f"https://business.financialpost.com/page/{n_of_page}?s={k_word}"
# https://financialpost.com/search/?search_text=microsoft&date_range=-30d&sort=desc
# https://financialpost.com/search/?search_text=microsoft&date_range=-30d&sort=desc&from=10
# https://financialpost.com/search/?search_text=microsoft&date_range=-30d&sort=desc&from=20
    link = f'https://financialpost.com/search/?search_text={k_word}&date_range=-30d&sort=desc&form={n_of_page}'
    print(f'Processing url: {link}')
    news_df["internal_urls"]  = get_links(link)
    news_df["principal_url"]  = link    
    news_df["n_of_page"]      = n_of_page
    print(f'news_df: {news_df}')
    return(news_df[["n_of_page","principal_url","internal_urls"]])

In [29]:
n_of_pages = 1
df = pd.concat([extract_link_of_news("microsoft",i) for i in range(n_of_pages)], ignore_index=True)
df

Processing url: https://financialpost.com/search/?search_text=microsoft&date_range=-30d&sort=desc&form=0
hrefs for https://financialpost.com/search/?search_text=microsoft&date_range=-30d&sort=desc&form=0:
[
    "/pmn/business-pmn/tiktoks-chinese-owner-offers-to-forego-stake-to-clinch-u-s-deal-sources-3/wcm/ac1a845a-7a8e-408c-85a0-bb265003144c/",
    "/pmn/business-pmn/tiktoks-chinese-owner-offers-to-forego-stake-to-clinch-u-s-deal-sources/wcm/733ca095-56b8-438e-a21e-f6759355e8c0/",
    "/pmn/business-pmn/trump-says-will-ban-tiktok-amid-pressure-on-chinese-owner-to-sell/wcm/d3e87f2f-0f8a-4006-b80b-ee0ac8df84e8/",
    "/pmn/business-pmn/nasdaq-surges-on-megacap-earnings-but-u-s-fiscal-uncertainty-nags-2/wcm/16fd8292-5fbe-41f0-9e07-d6c182f8d764/",
    "/pmn/business-pmn/bytedance-offers-to-forgo-stake-in-tiktok-to-clinch-u-s-deal-sources/wcm/8fe8cb2c-f37a-4261-9820-c49426a0a7bd/",
    "/pmn/business-pmn/microsoft-in-talks-to-buy-tiktok-nyt-2/wcm/624cc4f7-b796-4fe7-8fa5-7bcdf054f9b5/",
   

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

# 2. Extract the date and text from each link 

In [6]:
def extract_date_and_text(ur):
    url = ur
    
    try:
        soup         = BeautifulSoup(requests.get(url).content, 'html.parser')
        g = 0
    
    except:   
        the_type, the_value, the_traceback = sys.exc_info()
        g = 1
    

    if(g==0):

        if(soup.find("div",class_ = "entry-content").find_all("p")):
            text_of_news = soup.find("div",class_ = "entry-content").find_all("p")
        elif(soup.find("div",class_ = "entry-content").find("p")):
            text_of_news = soup.find("div",class_ = "entry-content").find("p")
        else:
            text_of_news = "NO TEXT"

        final_date   = soup.find("time")["datetime"]

        get_text_vec = [i.text for i in text_of_news]
        final_text = "".join(get_text_vec)
        
        
    else: final_date = "";final_text = ""
    
    return(final_date,final_text)

In [7]:
a = np.vectorize(extract_date_and_text)
test = df[0:10].copy()
test["internals_dates"],test["internals_text"] = a(test["internal_urls"][0:10])
test.head()

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

In [9]:
a = np.vectorize(extract_date_and_text)
df["internals_dates"],df["internals_text"] = a(df["internal_urls"])

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

In [10]:
df.head()

,n_of_page,principal_url,internal_urls,internals_dates,internals_text
0,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/pmn/busines...,2019-08-27T09:25:03-04:00,AMSTERDAM — Microsoft is remotely collecting d...
1,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/technology/...,2015-03-18T11:01:07-04:00,"ATLANTA, Georgia – This week, over 10,000 peop..."
2,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/pmn/press-r...,2018-09-25T09:02:40-04:00,"NEW YORK — Pexip, an award-winning video conf..."
3,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/business-in...,2012-11-13T12:02:39-05:00,Microsoft’s Windows boss Steven Sinofsky is ou...
4,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/business-in...,2013-04-15T11:56:02-04:00,Microsoft is working on a smartwatch with a 1....


In [11]:
df.loc[df["internals_text"] == "NO TEXT"]

,n_of_page,principal_url,internal_urls,internals_dates,internals_text


In [12]:
df.loc[df["internals_text"] == ""]

,n_of_page,principal_url,internal_urls,internals_dates,internals_text


In [14]:
df.to_pickle("data sets/microsoft_complete_text.pkl")